In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("kmeans").getOrCreate()

24/01/20 03:02:06 WARN Utils: Your hostname, Savvass-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.193.99 instead (on interface en0)
24/01/20 03:02:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/20 03:02:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
data = spark.read.csv("seeds_dataset.csv", header=True, inferSchema=True)

In [3]:
data.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)



## Transform and scale the data

In [4]:
from pyspark.ml.feature import VectorAssembler, StandardScaler

/Users/schanlar/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
assembler = VectorAssembler(inputCols=data.columns, outputCol='features')

In [6]:
final_data = assembler.transform(data).select('features')

In [7]:
final_data.printSchema()

root
 |-- features: vector (nullable = true)



In [8]:
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures')

scaler_model = scaler.fit(final_data)

final_data = scaler_model.transform(final_data)

## Creating a Clustering Model

In [9]:
from pyspark.ml.clustering import KMeans

In [10]:
kmeans = KMeans(featuresCol='scaledFeatures', k=3)

In [11]:
model = kmeans.fit(final_data)

24/01/20 03:02:48 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/01/20 03:02:48 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [12]:
# Make predictions 
predictions = model.transform(final_data)

In [13]:
predictions.show(5)

+--------------------+--------------------+----------+
|            features|      scaledFeatures|prediction|
+--------------------+--------------------+----------+
|[15.26,14.84,0.87...|[5.24452795332028...|         2|
|[14.88,14.57,0.88...|[5.11393027165175...|         2|
|[14.29,14.09,0.90...|[4.91116018695588...|         2|
|[13.84,13.94,0.89...|[4.75650503761158...|         2|
|[16.14,14.99,0.90...|[5.54696468981581...|         2|
+--------------------+--------------------+----------+
only showing top 5 rows



In [14]:
from pyspark.ml.evaluation import ClusteringEvaluator

In [15]:
# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)

print(f"Silhouette with squared euclidean distance : {silhouette:.3f}")

Silhouette with squared euclidean distance : 0.623


In [16]:
centers = model.clusterCenters()

In [17]:
# 3 groups of centers
centers

[array([ 6.32636687, 12.38115343, 37.39222755, 13.9206997 ,  9.75485787,
         2.41428142, 12.28078861]),
 array([ 4.06818854, 10.13938448, 35.87110297, 11.81191124,  7.52564313,
         3.24585755, 10.40780927]),
 array([ 4.90993613, 10.92295738, 37.28032496, 12.38401355,  8.5873381 ,
         1.7739463 , 10.35147469])]